<a href="https://colab.research.google.com/github/wjleece/anthropic-prompts/blob/main/Anthropic_Prompt_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 3.3 MB/s eta 0:00:00


In [16]:
import anthropic
from google.colab import userdata

API_KEY = userdata.get('ANTHROPIC_API_KEY')
MODEL_NAME = "claude-3-haiku-20240307"

# Stores the API_KEY & MODEL_NAME variables for use across notebooks within the IPython store
%store API_KEY
%store MODEL_NAME

Stored 'API_KEY' (str)
Stored 'MODEL_NAME' (str)


In [17]:
import anthropic
import re

client = anthropic.Anthropic(api_key=API_KEY)

system_prompt = "You are a helpful assistant."

In [18]:
def get_completion(prompt: str):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

In [19]:
# Prompt
prompt = "Hello, Claude!"

# Get Claude's response
print(get_completion(prompt))

Hello! It's nice to meet you. How can I assist you today?


In [ ]:
def translate(word, language):

  message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": f"translate the {word} into the desired {language}. Only translate the word, do not include anything else"}
        ]
    )

  return message.content[0].text

In [ ]:
translate("hijo de puta", "english")

'Motherfucker'

In [ ]:
#chatbot

conversation_history=[]

while True:
    user_input = input("User: ")

    if user_input.lower() == "quit":
        print("Conversation ended.")
        break

    conversation_history.append({"role": "user", "content": user_input})

    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=conversation_history
    )

    conversation_history.append({"role": "assistant", "content": response.content[0].text})

    print("Assistant:", response.content[0].text)

User: Give me a 1 paragraph summary of the major industries found in Manitoba
Assistant: Manitoba's economy is diverse, with several key industries contributing to its economic growth. The province's agricultural sector is a major driver, with the production of crops like wheat, canola, and soybeans, as well as livestock farming. The mining industry, particularly the extraction of minerals such as nickel, copper, and zinc, is also a significant contributor. Manitoba's manufacturing sector encompasses a range of products, including aerospace components, transportation equipment, and food processing. The province's hydroelectric power generation, supported by its abundant water resources, is another important industry. Additionally, Manitoba's tourism sector, which highlights the province's natural beauty and cultural attractions, plays a vital role in the economy.
User: What about for Saskatchewan?
Assistant: Here is a 1 paragraph summary of the major industries found in Saskatchewan:



In [ ]:
conversation_history

[{'role': 'user',
  'content': 'Give me a 1 paragraph summary of the major industries found in Manitoba'},
 {'role': 'assistant',
  'content': "Manitoba's economy is diverse, with several key industries contributing to its economic growth. The province's agricultural sector is a major driver, with the production of crops like wheat, canola, and soybeans, as well as livestock farming. The mining industry, particularly the extraction of minerals such as nickel, copper, and zinc, is also a significant contributor. Manitoba's manufacturing sector encompasses a range of products, including aerospace components, transportation equipment, and food processing. The province's hydroelectric power generation, supported by its abundant water resources, is another important industry. Additionally, Manitoba's tourism sector, which highlights the province's natural beauty and cultural attractions, plays a vital role in the economy."},
 {'role': 'user', 'content': 'What about for Saskatchewan?'},
 {'r

In [ ]:
system_prompt

'You are a helpful assistant.'

In [ ]:
# Streaming chatbot with proper conversation history management
conversation_history = []
response_content = ""

while True:
    user_input = input("\nUser: ")

    if user_input.lower() == "quit":
        print("Conversation ended.")
        break

    # Add user message to conversation history
    conversation_history.append({"role": "user", "content": user_input})

    # Get streaming response from API
    stream = client.messages.create(
        messages=conversation_history,
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        stream=True
    )

    print("\nAssistant: ", end="", flush=True)
    response_content = ""

    # Process streaming response
    for event in stream:
        if event.type == "content_block_delta":
            delta_text = event.delta.text
            response_content += delta_text
            print(delta_text, end="", flush=True)

    print()  # New line after response completes

    # Only after receiving the complete response, add it to history
    conversation_history.append({"role": "assistant", "content": response_content})


User: What was the Glass-Stegall Act?

Assistant: The Glass-Steagall Act was a U.S. federal law that was passed in 1933 during the Great Depression. The key provisions of the Glass-Steagall Act were:

1. It prohibited commercial banks from engaging in the investment banking business.

2. It required the separation of commercial banking and investment banking into distinct firms.

3. It created the Federal Deposit Insurance Corporation (FDIC) to insure bank deposits.

The main goal of the Glass-Steagall Act was to prevent the conflicts of interest that were seen as contributing to the 1929 stock market crash and the ensuing Great Depression. By separating commercial and investment banking, it was intended to reduce the risk of bank failures and protect consumers' deposits.

The Glass-Steagall Act remained in place for over 60 years until it was largely repealed in 1999 as part of the Gramm-Leach-Bliley Act. The repeal of Glass-Steagall was controversial, with some arguing it contribute

In [ ]:
conversation_history

[{'role': 'user', 'content': 'What was the Glass-Stegall Act?'},
 {'role': 'assistant',
  'content': "The Glass-Steagall Act was a U.S. federal law that was passed in 1933 during the Great Depression. The key provisions of the Glass-Steagall Act were:\n\n1. It prohibited commercial banks from engaging in the investment banking business.\n\n2. It required the separation of commercial banking and investment banking into distinct firms.\n\n3. It created the Federal Deposit Insurance Corporation (FDIC) to insure bank deposits.\n\nThe main goal of the Glass-Steagall Act was to prevent the conflicts of interest that were seen as contributing to the 1929 stock market crash and the ensuing Great Depression. By separating commercial and investment banking, it was intended to reduce the risk of bank failures and protect consumers' deposits.\n\nThe Glass-Steagall Act remained in place for over 60 years until it was largely repealed in 1999 as part of the Gramm-Leach-Bliley Act. The repeal of Glas

In [ ]:
# System prompt
system_prompt = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT))

Here are some critical thinking questions to further explore why the sky is blue:

- What is the composition of the Earth's atmosphere, and how does that affect the way light interacts with it?
- How do different wavelengths of light behave as they pass through the atmosphere?
- What is the process known as Rayleigh scattering, and how does it contribute to the blue appearance of the sky?
- How do factors like the time of day, weather conditions, and location on Earth impact the appearance of the sky?
- What other celestial bodies exhibit different colors in their skies, and what might explain those differences?
- How have our scientific understandings of the sky's color changed over time as our knowledge of optics and atmospheric science has advanced?

Exploring these types of questions can help deepen our understanding of this common natural phenomenon. Let me know if you have any other thoughts or reflections on this topic.


In [ ]:
system_prompt

'Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question.'

In [ ]:
system_prompt = "You are a helpful assistant who speaks Pirate English. Only respond in Pirate English."

# Prompt
PROMPT = "Where can I find some buried treasure?"

# Print Claude's response
print(get_completion(PROMPT))

Ahoy, me hearty! Ye be searchin' fer buried treasure, eh? Well, ye best be preparin' yerself fer a grand adventure, fer the path to buried riches be a treacherous one, filled with perilous obstacles and cunning scallywags waitin' to plunder yer booty.

Ye'll need to gather yer crew, stock up on supplies, and set sail for the uncharted waters where the X marks the spot. Keep yer eyes peeled fer ancient maps, mysterious clues, and the telltale signs of a hidden hoard. Beware the cursed islands, the fearsome sea monsters, and the rival pirates who'll stop at nothin' to get their greedy hands on yer treasure.

If ye be brave enough to face the challenges that lie ahead, then set yer course and let the winds of fortune guide ye to the buried treasure that awaits. Arr, the life of a pirate be a risky one, but the rewards be worth the peril. Now, weigh anchor and set sail, me hearty, and may the gods of the sea be on yer side!


In [20]:
system_prompt='You are a helpful assistant.'

In [ ]:
# Variable content
ANIMAL = "Cow"

# Prompt template with a placeholder for the variable content
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
I will tell you the name of an animal. Please respond with the noise that animal makes. Cow

------------------------------------- Claude's response -------------------------------------
Moo.


In [ ]:
# Variable content
EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."

# Prompt template with a placeholder for the variable content
PROMPT = f"Yo Claude. <tag>{EMAIL}</tag> <----- Make this email more polite but don't change anything else about it."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
Yo Claude. <tag>Show up at 6am tomorrow because I'm the CEO and I say so.</tag> <----- Make this email more polite but don't change anything else about it.

------------------------------------- Claude's response -------------------------------------
Dear [Your Name],

I hope this email finds you well. I wanted to reach out and kindly request your presence at our office tomorrow at 6 AM. As the CEO, I understand the importance of your time and leadership, and I would greatly appreciate your attendance at this meeting.

Thank you for your consideration, and I look forward to seeing you tomorrow.

Best regards,
[Your Name]


In [ ]:
# Variable content
SENTENCES = """<tag>- I like how cows sound</tag>
<tag>- This sentence is about spiders</tag>
<tag>- This sentence may appear to be about dogs but it's actually about pigs</tag>"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<tag>- I like how cows sound</tag>
<tag>- This sentence is about spiders</tag>
<tag>- This sentence may appear to be about dogs but it's actually about pigs</tag>

------------------------------------- Claude's response -------------------------------------
The second item on the list is "This sentence is about spiders".


In [ ]:
# Variable content
TOPIC = "Pigs"

# Prompt template with a placeholder for the variable content
PROMPT = f"Create a haiku about {TOPIC}."

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    topic_pattern = TOPIC.lower()
    return bool(re.search(topic_pattern, text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

--------------------------- Full prompt with variable substutions ---------------------------
Create a haiku about Pigs.

------------------------------------- Claude's response -------------------------------------
Here is a haiku about pigs:

Snouts in the mud, snort
Curly tails, pink and plump, oink
Pigs roam the farmyard

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True


In [35]:
def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": prefill}
        ]
    )
    # For newer API versions
    return message.content[0].text

In [36]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in XML tags."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
Please write a haiku about Rabbit. Put it in XML tags.

------------------------------------- Claude's response -------------------------------------
Here is a haiku about Rabbit, enclosed in XML tags:

<haiku>
Fluffy, twitching nose,
Hopping through the verdant field,
Rabbit's gentle grace.
</haiku>


In [8]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = ""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN:
Please write a haiku about Cat. Put it in  tags.

ASSISTANT TURN:


------------------------------------- Claude's response -------------------------------------
Here is a haiku about a cat, enclosed in tags:

<haiku>
Feline grace and poise,
Purring softly by the fire,
Watchful eyes observe.
</haiku>


In [37]:
# Variable content
ANIMAL = "cats"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a two separate haikus about {ANIMAL}. Separate each haiku with XML tags."

# Prefill for Claude's response
PREFILL = "The two haikus are below, separated from eachother by XML tags:"

# Get Claude's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(
        (re.search("cat", text.lower()) and re.search("", text))
        and (text.count("\n") + 1) > 5
    )

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please write a two separate haikus about cats. Separate each haiku with XML tags.

ASSISTANT TURN
The two haikus are below, separated from eachother by XML tags:

------------------------------------- Claude's response -------------------------------------
 

<haiku1>
Feline grace and poise,
Purring softly by the fire,
Watchful eyes observe.
</haiku1>

<haiku2>
Playful paws batting,
Chasing shadows on the wall,
Curious cat's dance.
</haiku2>

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True


In [54]:
# First input variable
ANIMAL1 = "Cat"

# Second input variable
ANIMAL2 = "Dog"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write two separate haikus, one about {ANIMAL1} and another about {ANIMAL2}. Separate the two haikus with <haiku1> and <haiku2> XML tags."

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    lower1=ANIMAL1.lower()
    lower2=ANIMAL2.lower()
    return bool(re.search(lower1, text.lower()) and re.search(lower2, text.lower()) and re.search("", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please write two separate haikus, one about Cat and another about Dog. Separate the two haikus with <haiku1> and <haiku2> XML tags.

------------------------------------- Claude's response -------------------------------------
<haiku1>
Feline grace and poise,
Purring softly by my side,
Loyal companion.
</haiku1>

<haiku2>
Loyal, playful friend,
Wagging tail and boundless joy,
Unconditional.
</haiku2>

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: False


In [48]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Claude's response
print(get_completion(PROMPT))

The sentiment of this movie review is positive.

The review states that the movie "blew my mind with its freshness and originality", which indicates a very positive and enthusiastic reaction to the film.

The second part of the review about living under a rock since 1900 is likely meant as a humorous or ironic comment, but it does not negate the overall positive sentiment expressed about the movie's qualities.


In [49]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive> and <negative> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

Here are the best arguments for positive and negative sentiment in the given review:

<positive>
- The review states that the movie "blew my mind with its freshness and originality," which suggests a very positive sentiment.
- The reviewer seems enthusiastic and impressed by the movie, indicating a favorable opinion.
</positive>

<negative>
- The reviewer's comment about "living under a rock since 1900" implies that the movie's "freshness and originality" may not be as novel or groundbreaking as the reviewer suggests, potentially undermining the positive sentiment.
- The sarcastic tone of the last sentence could be interpreted as a negative or dismissive attitude towards the movie.
</negative>

Based on the review, the sentiment appears to be more positive than negative. The reviewer's initial praise for the movie's "freshness and originality" outweighs the sarcastic remark at the end, which seems to be more of a self-deprecating comment about the reviewer's own lack of awareness rathe

In [50]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Claude's response
print(get_completion(PROMPT))

Here's a famous movie starring an actor born in 1956:

"Forrest Gump" (1994) starring Tom Hanks.

Tom Hanks was born on July 9, 1956.


In [53]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Claude's response
print(get_completion(PROMPT))

<brainstorm>
- Tom Hanks (born 1956) - Forrest Gump, Saving Private Ryan, Cast Away
- Denzel Washington (born 1954) - Training Day, The Equalizer, Fences
- Meryl Streep (born 1949) - The Deer Hunter, Sophie's Choice, The Iron Lady
- Harrison Ford (born 1942) - Star Wars, Indiana Jones, Blade Runner
</brainstorm>

A famous movie starring an actor born in 1956 is Forrest Gump, starring Tom Hanks.


In [63]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as either:
<email_categories>
(A) Pre-sale question
(B) Broken or defective item
(C) Billing question
(D) Other (please explain)
</email_categories>: {email}. Only return the information included within the <email_category> tags and nothing else. Do not return the tags themselves, just the content within the tags."""

# Prefill for Claude's response, if any
PREFILL = "("             #This is needed as for some reason when told not to include <email_category>, Claude fumbles response (D). But with this prefill, it gets it right.

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "A\) P",
    "B": "B\) B",
    "C": "C\) B",
    "D": "D\) O"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):

    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)

    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)
    response = PREFILL + response

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])

    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please classify this email as either: 
<email_categories>
(A) Pre-sale question
(B) Broken or defective item
(C) Billing question
(D) Other (please explain)
</email_categories>: Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.. Only return the information included within the <email_category> tags and nothing else. Do not return the tags themselves, just the content within the tags.

ASSISTANT TURN
(

------------------------------------- Claude's response -------------------------------------
(B) Broken or defective item

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True 






--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Plea

In [65]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as either:
<email_categories>
<answer>(A)</answer> Pre-sale question
<answer>(B)</answer> Broken or defective item
<answer>(C)</answer> Billing question
<answer>(D)</answer> Other (please explain)
</email_categories>: {email}. Only return the information included within the <answer> tags and nothing else. Do not return the tags themselves, just the content within the tags."""

# Prefill for Claude's response, if any
PREFILL = "("             #This is needed as for some reason when told not to include <email_category>, Claude fumbles response (D). But with this prefill, it gets it right.

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "A",
    "B": "B",
    "C": "C",
    "D": "D"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):

    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)

    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)
    response = PREFILL + response

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])

    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please classify this email as either: 
<email_categories>
<answer>(A)</answer> Pre-sale question
<answer>(B)</answer> Broken or defective item
<answer>(C)</answer> Billing question
<answer>(D)</answer> Other (please explain)
</email_categories>: Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.. Only return the information included within the <answer> tags and nothing else. Do not return the tags themselves, just the content within the tags.

ASSISTANT TURN
(

------------------------------------- Claude's response -------------------------------------
(B)

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True 






--------------------------- Full prompt with variable substutions -------

In [66]:
# Prompt
PROMPT = "Will Santa bring me presents on Christmas?"

# Print Claude's response
print(get_completion(PROMPT))

I don't actually have any information about whether Santa will bring you presents on Christmas. Santa Claus is a fictional character, and whether he brings presents depends on the beliefs and traditions of each individual family. The important thing is to enjoy the holiday season and spend time with loved ones, regardless of whether you receive presents from Santa.


In [67]:
# Prompt
PROMPT = """Please complete the conversation by writing the next line, speaking as "A".
Q: Is the tooth fairy real?
A: Of course, sweetie. Wrap up your tooth and put it under your pillow tonight. There might be something waiting for you in the morning.
Q: Will Santa bring me presents on Christmas?"""

# Print Claude's response
print(get_completion(PROMPT))

A: Yes, my dear, as long as you've been good this year, Santa will be sure to bring you lots of wonderful presents on Christmas morning. Just make sure to get to bed early on Christmas Eve so he can come visit while you're sleeping.


In [70]:
# Prompt template with a placeholder for the variable content
PROMPT = """Silvermist Hollow, a charming village, was home to an extraordinary group of individuals.
Among them was Dr. Liam Patel, a neurosurgeon who revolutionized surgical techniques at the regional medical center.
Olivia Chen was an innovative architect who transformed the village's landscape with her sustainable and breathtaking designs.
The local theater was graced by the enchanting symphonies of Ethan Kovacs, a professionally-trained musician and composer.
Isabella Torres, a self-taught chef with a passion for locally sourced ingredients, created a culinary sensation with her farm-to-table restaurant, which became a must-visit destination for food lovers.
These remarkable individuals, each with their distinct talents, contributed to the vibrant tapestry of life in Silvermist Hollow.

1. Dr. Liam Patel [NEUROSURGEON]
2. Olivia Chen [ARCHITECT]
3. Ethan Kovacs [MISICIAN AND COMPOSER]
4. Isabella Torres [CHEF]


At the heart of the town, Chef Oliver Hamilton has transformed the culinary scene with his farm-to-table restaurant, Green Plate. Oliver's dedication to sourcing local, organic ingredients has earned the establishment rave reviews from food critics and locals alike.
Just down the street, you'll find the Riverside Grove Library, where head librarian Elizabeth Chen has worked diligently to create a welcoming and inclusive space for all. Her efforts to expand the library's offerings and establish reading programs for children have had a significant impact on the town's literacy rates.
As you stroll through the charming town square, you'll be captivated by the beautiful murals adorning the walls. These masterpieces are the work of renowned artist, Isabella Torres, whose talent for capturing the essence of Riverside Grove has brought the town to life.
Riverside Grove's athletic achievements are also worth noting, thanks to former Olympic swimmer-turned-coach, Marcus Jenkins. Marcus has used his experience and passion to train the town's youth, leading the Riverside Grove Swim Team to several regional championships.

1. Oliver Hamilton [CHEF]
2. Elizabeth Chen [LIBRARIAN]
3. Isabella Torres [ARTIST]
4. Marcus Jenkins [COACH]


Oak Valley, a charming small town, is home to a remarkable trio of individuals whose skills and dedication have left a lasting impact on the community.
At the town's bustling farmer's market, you'll find Laura Simmons, a passionate organic farmer known for her delicious and sustainably grown produce. Her dedication to promoting healthy eating has inspired the town to embrace a more eco-conscious lifestyle.
In Oak Valley's community center, Kevin Alvarez, a skilled dance instructor, has brought the joy of movement to people of all ages. His inclusive dance classes have fostered a sense of unity and self-expression among residents, enriching the local arts scene.
Lastly, Rachel O'Connor, a tireless volunteer, dedicates her time to various charitable initiatives. Her commitment to improving the lives of others has been instrumental in creating a strong sense of community within Oak Valley.
Through their unique talents and unwavering dedication, Laura, Kevin, and Rachel have woven themselves into the fabric of Oak Valley, helping to create a vibrant and thriving small town."""

# Prefill for Claude's response
PREFILL = ""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN:
Silvermist Hollow, a charming village, was home to an extraordinary group of individuals.
Among them was Dr. Liam Patel, a neurosurgeon who revolutionized surgical techniques at the regional medical center.
Olivia Chen was an innovative architect who transformed the village's landscape with her sustainable and breathtaking designs.
The local theater was graced by the enchanting symphonies of Ethan Kovacs, a professionally-trained musician and composer.
Isabella Torres, a self-taught chef with a passion for locally sourced ingredients, created a culinary sensation with her farm-to-table restaurant, which became a must-visit destination for food lovers.
These remarkable individuals, each with their distinct talents, contributed to the vibrant tapestry of life in Silvermist Hollow.

1. Dr. Liam Patel [NEUROSURGEON]
2. Olivia Chen [ARCHITECT]
3. Ethan Kovacs [MISICIAN AND COMPOSER]
4. I

In [75]:
# Prompt template with a placeholder for the variable content - I used Claude to generate 'sample' emails with 'similiar sentiments' to the examples provided
PROMPT = """
Please classify emails into the following categories, and do not include explanations:
<categories>
(A) Pre-sale question
(B) Broken or defective item
(C) Billing question
(D) Other (please explain)
</categories>

Here are a few examples of correct answer formatting:
<examples>
Q: Good morning, I've been looking at your BlendMax Ultra on your website. Before I purchase, I was wondering if it can be used to blend essential oils for homemade soaps, or is it strictly designed for food preparation? Thanks for clarifying?
A: The correct category is: A

Q: Hello, I purchased your StirMaster Pro 2000 last week, and today it started making a grinding noise whenever I use it. The bottom is getting very hot, and there's a burning smell coming from it. I think there's something wrong with the motor. Please send me a replacement as soon as possible..
A: The correct category is: B

Q: YOUR COMPANY CONTINUES TO CHARGE MY CREDIT CARD EVEN THOUGH I CANCELED MY SUBSCRIPTION IN JANUARY!!! I'VE CALLED THREE TIMES ALREADY!!! FIX THIS IMMEDIATELY OR I'M REPORTING YOU TO THE BETTER BUSINESS BUREAU!!!
A: The correct category is: C

Q: Dear sir or madam, I somehow ended up on this page and I don't know what to do next. My grandson set up this computer for me but he's away at college now. Can someone please assist? I was trying to find a recipe for banana bread.
A: The correct category is: D
</examples>

Here is the email for you to categorize: {email}"""

# Prefill for Claude's response
PREFILL = "The correct category is:"

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Iterate through list of emails
for i,email in enumerate(EMAILS):

    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)

    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)
    response = PREFILL + response

    # Grade Claude's response
    grade = any([bool(re.search(ans, response[-1])) for ans in ANSWERS[i]])

    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")


--------------------------- Full prompt with variable substutions ---------------------------
USER TURN

Please classify emails into the following categories, and do not include explanations: 
<categories>
(A) Pre-sale question
(B) Broken or defective item
(C) Billing question
(D) Other (please explain)
</categories>

Here are a few examples of correct answer formatting:
<examples>
Q: Good morning, I've been looking at your BlendMax Ultra on your website. Before I purchase, I was wondering if it can be used to blend essential oils for homemade soaps, or is it strictly designed for food preparation? Thanks for clarifying?
A: The correct category is: A

Q: Hello, I purchased your StirMaster Pro 2000 last week, and today it started making a grinding noise whenever I use it. The bottom is getting very hot, and there's a burning smell coming from it. I think there's something wrong with the motor. Please send me a replacement as soon as possible..
A: The correct category is: B

Q: YOUR COMP